In [5]:
!pip3 install nltk

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk

In [7]:
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""


In [8]:
# Tokenization
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /Users/abroadhub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/abroadhub/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [11]:
# convert word to Token
tokens = nltk.word_tokenize(document.lower())

In [16]:
# Build Vocabulary
vocab = {'<UNK>':0}

Counter(tokens) # check the each token repeated how many times
# Counter(tokens).keys() # print the all uniquie words

for token in Counter(tokens).keys():
    #print(token)
    if token not in vocab:
        vocab[token] = len(vocab)

print(vocab)

{'<UNK>': 0, 'about': 1, 'the': 2, 'program': 3, 'what': 4, 'is': 5, 'course': 6, 'fee': 7, 'for': 8, 'data': 9, 'science': 10, 'mentorship': 11, '(': 12, 'dsmp': 13, '2023': 14, ')': 15, 'follows': 16, 'a': 17, 'monthly': 18, 'subscription': 19, 'model': 20, 'where': 21, 'you': 22, 'have': 23, 'to': 24, 'make': 25, 'payments': 26, 'of': 27, 'rs': 28, '799/month': 29, '.': 30, 'total': 31, 'duration': 32, '?': 33, '7': 34, 'months': 35, 'so': 36, 'becomes': 37, '799': 38, '*': 39, '=': 40, '5600': 41, 'approx': 42, 'syllabus': 43, 'we': 44, 'will': 45, 'be': 46, 'covering': 47, 'following': 48, 'modules': 49, ':': 50, 'python': 51, 'fundamentals': 52, 'libraries': 53, 'analysis': 54, 'sql': 55, 'maths': 56, 'machine': 57, 'learning': 58, 'ml': 59, 'algorithms': 60, 'practical': 61, 'mlops': 62, 'case': 63, 'studies': 64, 'can': 65, 'check': 66, 'detailed': 67, 'here': 68, '-': 69, 'https': 70, '//learnwith.campusx.in/courses/campusx-data-science-mentorship-program-637339afe4b0615a1bbed

In [17]:
len(vocab)

289

In [24]:
#  Extract sentences from data
input_sentences = document.split('\n')
print(input_sentences)

['About the Program', 'What is the course fee for  Data Science Mentorship Program (DSMP 2023)', 'The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.', 'What is the total duration of the course?', 'The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)', 'What is the syllabus of the mentorship program?', 'We will be covering the following modules:', 'Python Fundamentals', 'Python libraries for Data Science', 'Data Analysis', 'SQL for Data Science', 'Maths for Machine Learning', 'ML Algorithms', 'Practical ML', 'MLOPs', 'Case studies', 'You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390', 'Will Deep Learning and NLP be a part of this program?', 'No, NLP and Deep Learning both are not a part of this program’s curriculum.', 'What if I miss a live session? Will I get a recording of the session?', '